In [1]:
import scipy.io as scio
import numpy as np
import os 
from glob import *
import nibabel as nib
import pandas as pd
import random
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.pipeline import make_pipeline
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.decomposition import PCA

In [57]:
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.manifold import TSNE
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.feature_selection import mutual_info_classif
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

In [3]:
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import VotingClassifier

In [90]:
f_state = np.loadtxt(r'H:\Matlab\Work\Age\Gift_FCM\ICA\DFNC\TR_30\frac_time_state.txt')
mdt = np.loadtxt(r'H:\Matlab\Work\Age\Gift_FCM\ICA\DFNC\TR_30\mean_dwell_time.txt')
nt = np.loadtxt(r'H:\Matlab\Work\Age\Gift_FCM\ICA\DFNC\TR_30\num_transitions.txt')
transition_m_path = r'H:\Matlab\Work\Age\Gift_FCM\ICA\DFNC\TR_30\transition_matrix'
subj_tm = []
for path in os.listdir(transition_m_path):
    if path.startswith('Sub'):
#         print(os.path.join(transition_m_path,path))
        tm = np.loadtxt(os.path.join(transition_m_path,path))
        subj_tm.append(tm.flatten())

In [217]:
state_featue = {}
# for i in range(4):
#     fname = 'fraction'+str(i)
#     state_featue[fname] = np.expand_dims(np.array(f_state[:,i]),1)
# for i in range(4):
#     mdtname = 'MDT'+str(i)
#     state_featue[mdtname] = np.expand_dims(np.array(mdt[:,i]),1)

# state_featue['numoftans'] = np.expand_dims(np.array(nt),1)
# for i in range(16):
#     tansition_name = 'tm'+str(i)
#     state_featue[f'{tansition_name}'] = np.expand_dims(np.array(subj_tm[0]),1)
    

state_featue['numoftans'] = np.expand_dims(np.array(nt),1)
state_featue['fraction'] = np.array(f_state)
state_featue['MDT'] =np.array(mdt)
state_featue['transitionP'] =np.array(subj_tm)

In [150]:
# for i in state_featue.keys():
#     print(i)
# f_state.flatten().shape
# state_featue['fraction0'].shape
# state_featue
# state_featue['numoftans'].shape

(62, 1)

In [218]:
state_fea_result = {}
for fea_name in state_featue.keys():
    feature_val = state_featue[fea_name]
#     c = np.nan_to_num(test)
    feature_val_sff, y_data = shuffle_data(feature_val,y_datas)
#     sk_std = StandardScaler()
#     new_degraph = sk_std.fit_transform(dgraph)
#     train_ydata = y_data[:51]
#     test_ydata = y_data[51:]
#     train_xdata,test_xdata = get_train_test_data(dgraph)
#     with open('state_log.txt','a+') as rf:
#         rf.write(f"\n #################state feature############ \n")
    state_fea_result[fea_name] = []
    for name, model in zip(names, classifiers):
                scores = []
                try:
#                         model.fit(train_xdata, train_ydata)
        #             score = model.score(test_xdata, test_ydata)
                    score = cross_val_score(model,feature_val_sff,y_data,cv = 10)
                except Exception:
                    pass
#                 print(f'{name} ==> Mean：{np.mean(score)} ==> Std:{np.std(score)}')
                scores.append(np.mean(score))
                state_fea_result[fea_name].append(str(np.mean(score))+'+-'+str(np.std(score))) 
#                     rf.write(f"==> ==> {name}：, Mean: {np.mean(score)}, Std：{np.std(score)}\n")
    estimators = [(name, model) for name, model in zip(names, classifiers)]
    eclf1 = VotingClassifier(estimators = estimators, voting='hard')
    score = cross_val_score(eclf1, feature_val_sff, y_data,cv = 10)
    state_fea_result[fea_name].append(str(np.mean(score))+'+-'+str(np.std(score)))
#     print(state_fea_result)
#         rf.close()

D:\Anaconda3\lib\site-packages\sklearn\svm\_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
D:\Anaconda3\lib\site-packages\sklearn\svm\_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
D:\Anaconda3\lib\site-packages\sklearn\svm\_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
D:\Anaconda3\lib\site-packages\sklearn\svm\_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
D:\Anaconda3\lib\site-packages\sklearn\svm\_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
D:\Anaconda3\lib\site-packages\skle

In [221]:
state_fea_result
# {'numoftans': '0.54285714285714280.17195685795493132',
#  'fraction': '0.83809523809523810.12945025922922082',
#  'MDT': '0.58333333333333330.11627728997214076'}
state_fea_result_df=pd.DataFrame(state_fea_result)
# state_fea_result_df_t = pd.DataFrame(state_fea_result_df.values.T,index = state_fea_result_df.columns, columns =state_fea_result_df.index)
state_fea_result_df.index =pd.Series(c)
state_fea_result_df.to_excel('state_fea_result_df.xlsx')

In [212]:
c = ['Nearest Neighbors', 'Linear SVM', 'RBF SVM', 'Gaussian Process', 'Decision Tree', 'Random Forest', 'Neural Net', 'AdaBoost','Voting']
print(c)


['Nearest Neighbors', 'Linear SVM', 'RBF SVM', 'Gaussian Process', 'Decision Tree', 'Random Forest', 'Neural Net', 'AdaBoost', 'Voting']


In [214]:
state_concatfea_result ={}
state_concatfea_result['state_feature'] = []

concat_state_featue = state_featue['fraction']
print(concat_state_featue.shape)
concat_state_featue = np.concatenate([concat_state_featue,state_featue['MDT']],1)
print(concat_state_featue.shape)
concat_state_featue = np.concatenate([concat_state_featue,state_featue['numoftans']],1)
# concat_state_featue = np.array(concat_state_featue)
concat_state_featue.shape
concat_state_featue_sff, y_data = shuffle_data(concat_state_featue,y_datas)
for name, model in zip(names, classifiers):
            scores = []
            try:
#                         model.fit(train_xdata, train_ydata)
    #             score = model.score(test_xdata, test_ydata)
                score = cross_val_score(model,concat_state_featue_sff,y_data,cv = 10)
            except Exception:
                pass
            print(f'{name} ==> Mean：{np.mean(score)} ==> Std:{np.std(score)}')
            scores.append(np.mean(score))
            state_concatfea_result['state_feature'].append(str(np.mean(score))+'+-'+str(np.std(score)))
estimators = [(name, model) for name, model in zip(names, classifiers)]
eclf1 = VotingClassifier(estimators = estimators, voting='hard')
score = cross_val_score(eclf1, concat_state_featue_sff, y_data,cv = 10)
state_concatfea_result.append(str(np.mean(score))+'+-'+str(np.std(score)))

(62, 4)
(62, 20)
Nearest Neighbors ==> Mean：0.7952380952380953 ==> Std:0.20039643250000774
Linear SVM ==> Mean：0.8404761904761905 ==> Std:0.12374082343034556
RBF SVM ==> Mean：0.7404761904761904 ==> Std:0.18579058261808956


D:\Anaconda3\lib\site-packages\sklearn\svm\_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
D:\Anaconda3\lib\site-packages\sklearn\svm\_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
D:\Anaconda3\lib\site-packages\sklearn\svm\_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
D:\Anaconda3\lib\site-packages\sklearn\svm\_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
D:\Anaconda3\lib\site-packages\sklearn\svm\_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
D:\Anaconda3\lib\site-packages\skle

Gaussian Process ==> Mean：0.7285714285714284 ==> Std:0.15836465855538945
Decision Tree ==> Mean：0.8404761904761905 ==> Std:0.14445534582969058
Random Forest ==> Mean：0.8523809523809524 ==> Std:0.15714285714285717
Neural Net ==> Mean：0.8071428571428572 ==> Std:0.16058636884683095
AdaBoost ==> Mean：0.8547619047619047 ==> Std:0.1381978272469662


D:\Anaconda3\lib\site-packages\sklearn\svm\_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
D:\Anaconda3\lib\site-packages\sklearn\svm\_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
D:\Anaconda3\lib\site-packages\sklearn\svm\_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
D:\Anaconda3\lib\site-packages\sklearn\svm\_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
D:\Anaconda3\lib\site-packages\sklearn\svm\_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
D:\Anaconda3\lib\site-packages\skle

In [216]:
# state_concatfea_result
state_concatfea_result_df=pd.DataFrame(state_concatfea_result)
# state_fea_result_df_t = pd.DataFrame(state_fea_result_df.values.T,index = state_fea_result_df.columns, columns =state_fea_result_df.index)
state_concatfea_result_df.index =pd.Series(c)
state_concatfea_result_df.to_excel('state_concatfea_result_df.xlsx')

In [161]:
state_concatfea_result

['0.7880952380952381+-0.132330411729431',
 '0.8523809523809524+-0.11654512619543256',
 '0.8023809523809524+-0.1460166486329711',
 '0.7238095238095238+-0.13136299261079737',
 '0.8714285714285716+-0.11770673423150738',
 '0.869047619047619+-0.06585388898066347',
 '0.838095238095238+-0.10583862271853747']

In [4]:
age = pd.read_excel(r'H:\Jupyter\Age\age.xlsx')
y_datas = age['label'].tolist()
y_datas = np.array(y_datas)
ages =np.array(age['Age_Years']) 

In [5]:
root_path = r'H:\Matlab\Work\Age\Gift_FCM\ICA\DFNC\TR_30\Dgraph'
# state_name = ['State1','State2','State3','State4']
# NMN = ['gEfficiency','LocalEfficiency','Synchronization','Hierarchy','NumModularity','ModularityQ',
#        'ShorestPath','ClusteringCOff','EdgeBetwenness'] 
# NMN = ['ShorestPath','ClusteringCOff'] 
NMN = ['Assortativity','R_Assortativity','Synchronization','R_Synchronization','Hierarchy','R_Hierarchy',
       'gEfficiency','ModularityQ','R_ModularityQ','NumModularity',
       'R_NumModularity','Gamma','Lambda','Sigma','ClusteringCOff',
       'ShorestPath','LocalEfficiency','Volunerable','EdgeBetwenness']
# fig = plt.figure(figsize=(18, 12), facecolor='white')
# for state in state_name:
subjs = []
for i in range(62):
    if i+1<10:
        subjs.append(f'sub_00{i+1}')
    else:
        subjs.append(f'sub_0{i+1}')

subj_graph ={} 
# file.write(f'********************* {state}*****************\n')
for num,nmn in enumerate(NMN):
#     print(nmn)
#     with open(f'./state_top/dgraph/tp_dgraph.txt','a+') as file:
        subj_graph[nmn] =[]
        for subj_name in subjs:
    #         print(nmn)
            nam_path = os.path.join(root_path,subj_name,nmn+'.txt')
    #         print(nam_path)
            graph_measrue = np.loadtxt(nam_path) 
    #         print(graph_measrue.shape)
    #             file.writelines(f'{graph_measrue} \n')
            subj_graph[nmn].append(graph_measrue) 

In [10]:
# subj_graph
np.random.seed(0)
def shuffle_data(x_data,y_data):
#     print(x_data.shape)
    index = [i for i in range(len(x_data))] 
    random.shuffle(index)
    x_data = x_data[index]
    y_data = y_data[index]
    return x_data,y_data

## 属于 所得的 图指标得到 train和test data
def get_train_test_data(graph_measure):
    return graph_measure[:42],graph_measure[42:]

In [7]:
y_datas

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [9]:
test  = np.array(subj_graph['Assortativity'])

In [ ]:
# np.array(test).shape

In [ ]:
dgraph

In [10]:
dgraph, y_data = shuffle_data(test,y_datas)
train_ydata = y_data[:42]
test_ydata = y_data[42:]
# train_xdata,test_xdata = get_train_test_data(test)

In [11]:
train_ydata

array([1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0,
       1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0])

In [17]:
svm = SVC(gamma='auto',probability =True, verbose = 1)
# print(svm.fit(train_xdata,train_ydata))
# # # evlaution
# print(svm.score(test_xdata,test_ydata)) # mean acc
# print(svm.classes_)
# print(svm.decision_function(test_xdata))

print(cross_val_score(svm,dgraph,y_data, cv=3))

[LibSVM][LibSVM][LibSVM][0.42857143 0.61904762 0.45      ]


In [ ]:
#  # just plot the dataset first
# tsne_x =  TSNE(n_components=2,
#                   random_state = 0).fit_transform(test)

# colour = [ "red", "blue"]
# # colour = [ "red", "blue", "green", "yellow", "purple", "orange", "black",\
# #           '#FFB6C1','#4B0082','#778899','#E1FFFF','#8FBC8F','#808000','#FFEFD5','#CD853F' ]
# color = list(map(lambda x: colour[x], y_data))

# cm = plt.cm.RdBu
# plt.figure()

# plt.title("Input data")
# # Plot the training points
# plt.scatter(tsne_x[:, 0], tsne_x[:, 1], c= y_data, edgecolors="k")
# # # Plot the testing points
# # ax.scatter(
# #     X_test[:, 0], X_test[:, 1], c=y_test, cmap=cm_bright, alpha=0.6, edgecolors="k"
# # )

In [204]:
names = [
    "Nearest Neighbors",
    "Linear SVM",
    "RBF SVM",
    "Gaussian Process",
    "Decision Tree",
    "Random Forest",
    "Neural Net",
    "AdaBoost",
    "Naive Bayes",
    "QDA",
    "AdaBoost"
]


K= 8
selector = mutual_info_classif
# f_classif
feature_selection_classifiers  = [
    make_pipeline(SelectKBest(selector, k = K),MinMaxScaler(),KNeighborsClassifier(3)),
    make_pipeline(SelectKBest(selector, k = K),MinMaxScaler(),LinearSVC(random_state=0)),
    make_pipeline(SelectKBest(selector, k = K),MinMaxScaler(),SVC(gamma = 2, C=1,random_state=0)),
    make_pipeline(SelectKBest(selector, k = K),MinMaxScaler(),GaussianProcessClassifier(1.0 * RBF(1.0),random_state=0)),
    make_pipeline(SelectKBest(selector, k = K),MinMaxScaler(),DecisionTreeClassifier(max_depth=5,random_state=0)),
    make_pipeline(SelectKBest(selector, k = K),MinMaxScaler(),RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1,random_state=0)),
    make_pipeline(SelectKBest(selector, k = K),MinMaxScaler(),MLPClassifier(alpha=1, max_iter=2000,random_state=0)),
    make_pipeline(SelectKBest(selector, k = K),MinMaxScaler(),AdaBoostClassifier(random_state=0)),
    make_pipeline(SelectKBest(selector, k = K),MinMaxScaler(),GaussianNB()),
    make_pipeline(SelectKBest(selector, k = K),MinMaxScaler(),QuadraticDiscriminantAnalysis()),
]

classifiers = [
    KNeighborsClassifier(5),
    LinearSVC(random_state=0),
    SVC(gamma = 2, C=1,random_state=0),
    GaussianProcessClassifier(1.0 * RBF(1.0),random_state=0),
    DecisionTreeClassifier(max_depth=5,random_state=0),
    RandomForestClassifier(max_depth=5, n_estimators= 10, max_features=1,random_state=0),
    MLPClassifier(alpha=1, max_iter=2000,random_state=0),
    AdaBoostClassifier(random_state=0),
#     GaussianNB(),
#     QuadraticDiscriminantAnalysis(),
#     AdaBoostClassifier(n_estimators=100)
]

In [ ]:
train_xdata.shape
train_ydata.shape

In [ ]:
NMN_cat = ['R_Assortativity','R_Synchronization','Hierarchy','R_Hierarchy',
       'gEfficiency','ModularityQ','R_ModularityQ','NumModularity',
       'R_NumModularity','Gamma','Lambda','Sigma','ClusteringCOff',
       'ShorestPath','LocalEfficiency','Volunerable','EdgeBetwenness']

In [238]:
result = {}
for nmn in NMN:
    test = np.array(subj_graph[nmn])
    c = np.nan_to_num(test)
    dgraph, y_data = shuffle_data(c,y_datas)
    sk_std = StandardScaler()
    new_degraph = sk_std.fit_transform(dgraph)
#     train_ydata = y_data[:51]
#     test_ydata = y_data[51:]
#     train_xdata,test_xdata = get_train_test_data(dgraph)
    result[nmn] =[]
    for name, model in zip(names, classifiers):
                scores = []
                try:
#                         model.fit(train_xdata, train_ydata)
        #             score = model.score(test_xdata, test_ydata)
                    score = cross_val_score(model,new_degraph,y_data,cv = 10)
                except Exception:
                    pass
                print(f'{name} ==> Mean：{np.mean(score)} ==> Std:{np.std(score)}')
                scores.append(np.mean(score))
                result[nmn].append(str(np.mean(score))+'+-'+str(np.std(score)))
    estimators = [(name, model) for name, model in zip(names, classifiers)]
    eclf1 = VotingClassifier(estimators = estimators, voting='hard')
    score = cross_val_score(eclf1, concat_state_featue_sff, y_data,cv = 10)
    result[nmn].append(str(np.mean(score))+'+-'+str(np.std(score)))

In [239]:
# result

In [236]:
result_df=pd.DataFrame(result)
c = ['Nearest Neighbors', 'Linear SVM', 'RBF SVM', 'Gaussian Process', 'Decision Tree', 'Random Forest', 'Neural Net', 'AdaBoost','Voting']
print(c)
# state_fea_result_df_t = pd.DataFrame(state_fea_result_df.values.T,index = state_fea_result_df.columns, columns =state_fea_result_df.index)
result_df.index = pd.Series(c)
result_df.to_excel('result_df.xlsx')

['Nearest Neighbors', 'Linear SVM', 'RBF SVM', 'Gaussian Process', 'Decision Tree', 'Random Forest', 'Neural Net', 'AdaBoost', 'Voting']


In [235]:
result_df

,Assortativity,R_Assortativity,Synchronization,R_Synchronization,Hierarchy,R_Hierarchy,gEfficiency,ModularityQ,R_ModularityQ,NumModularity,R_NumModularity,Gamma,Lambda,Sigma,ClusteringCOff,ShorestPath,LocalEfficiency,Volunerable,EdgeBetwenness
Nearest Neighbors,0.5738095238095238+-0.1920913662050472,0.38333333333333336+-0.13403302804797684,0.3928571428571429+-0.1756910553749835,0.4809523809523809+-0.20887344951867917,0.5142857142857143+-0.1690979133768067,0.5714285714285714+-0.18257418583505539,0.5261904761904762+-0.20768948128136058,0.5333333333333332+-0.12874767461519787,0.45714285714285713+-0.1549632390025998,0.5619047619047619+-0.20039643250000774,0.6285714285714286+-0.202926435739643,0.488095238095238+-0.21593282318115542,0.5857142857142856+-0.20898198340468277,0.6119047619047617+-0.10965313275875671,0.45+-0.17051675495041788,0.530952380952381+-0.1862781428006538,0.5047619047619046+-0.1652319197418807,0.44761904761904764+-0.2541715456206194,0.5285714285714286+-0.19395630337539335
Linear SVM,0.5785714285714285+-0.11321422312351183,0.45714285714285713+-0.21157009420498152,0.3904761904761905+-0.24074713637646739,0.5023809523809524+-0.16129085289942804,0.6261904761904762+-0.11271238455756785,0.5095238095238096+-0.23138153549059698,0.5547619047619048+-0.19548474547720013,0.5642857142857143+-0.21952587541637153,0.5452380952380952+-0.24174582330187264,0.6928571428571428+-0.1331844406756857,0.6142857142857143+-0.16385381461033577,0.46904761904761905+-0.17497975263498217,0.5666666666666667+-0.19512191665283696,0.45476190476190476+-0.15,0.48571428571428577+-0.16468206157895796,0.5642857142857143+-0.1646304182196431,0.5166666666666667+-0.14208122616986377,0.530952380952381+-0.2416520051850953,0.5+-0.03194382824999699
RBF SVM,0.5476190476190477+-0.06563832739090579,0.5476190476190477+-0.06563832739090579,0.5476190476190477+-0.06563832739090579,0.5476190476190477+-0.06563832739090579,0.5476190476190477+-0.06563832739090579,0.5476190476190477+-0.06563832739090579,0.5476190476190477+-0.06563832739090579,0.5476190476190477+-0.06563832739090579,0.5476190476190477+-0.06563832739090579,0.5476190476190477+-0.06563832739090579,0.5476190476190477+-0.06563832739090579,0.5476190476190477+-0.06563832739090579,0.5476190476190477+-0.06563832739090579,0.5476190476190477+-0.06563832739090579,0.5476190476190477+-0.06563832739090579,0.5476190476190477+-0.06563832739090579,0.5476190476190477+-0.06563832739090579,0.5476190476190477+-0.06563832739090579,0.5476190476190477+-0.06563832739090579
Gaussian Process,0.4619047619047619+-0.2694160982822026,0.33809523809523806+-0.15275252316519466,0.42857142857142855+-0.2435098604865828,0.46904761904761905+-0.2174501793488941,0.5595238095238095+-0.20349831602706284,0.6071428571428571+-0.1405568775537661,0.4714285714285714+-0.27121963554091705,0.6166666666666667+-0.11139702443132939,0.5666666666666667+-0.22832327799498767,0.5976190476190476+-0.12254389745197876,0.5142857142857142+-0.12652695481510878,0.5452380952380952+-0.1301708279317776,0.5047619047619047+-0.2158146513071364,0.49523809523809526+-0.1474653577828764,0.44761904761904764+-0.12224283479041367,0.5+-0.1524553389864964,0.5309523809523811+-0.15358527069400132,0.44761904761904764+-0.20669080394486383,0.5976190476190477+-0.10457233931427501
Decision Tree,0.5642857142857143+-0.1775526494047825,0.6071428571428571+-0.14055687755376609,0.5047619047619047+-0.20968608434575237,0.4071428571428572+-0.17217101133134216,0.569047619047619+-0.1909072503371533,0.5761904761904761+-0.17228621442976708,0.34047619047619043+-0.15468862734076938,0.4833333333333333+-0.16837559245940137,0.5547619047619048+-0.2220932165226953,0.5785714285714285+-0.20148259998133364,0.5619047619047619+-0.17043362064926937,0.419047619047619+-0.17855555485002547,0.5904761904761904+-0.2192028435520744,0.5+-0.20619652471058064,0.4357142857142858+-0.1428769827493095,0.419047619047619+-0.18170270503179917,0.5666666666666667+-0.18832114222176372,0.7071428571428571+-0.14717675676691042,0.5666666666666667+

In [232]:
c = ['Assortativity',
 'R_Assortativity',
 'Synchronization',
 'R_Synchronization',
 'Hierarchy',
 'R_Hierarchy',
 'gEfficiency',
 'ModularityQ',
 'R_ModularityQ',
 'NumModularity',
 'R_NumModularity',
 'Gamma',
 'Lambda',
 'Sigma',
 'ClusteringCOff',
 'ShorestPath',
 'LocalEfficiency',
 'Volunerable',
 'EdgeBetwenness','Voting']

In [243]:
NMN_cat = ['Hierarchy',
       'gEfficiency','ModularityQ','NumModularity',
       'Gamma','Lambda','ClusteringCOff',
       'ShorestPath','LocalEfficiency','EdgeBetwenness']

In [244]:
concenate_graph =subj_graph ['Synchronization']
for nmn in NMN_cat:
    test = np.array(subj_graph[nmn])
    
    concenate_graph = np.concatenate([concenate_graph,test],1)
#     print(concenate_graph.shape)
    
#     c = np.nan_to_num(test)
#     dgraph, y_data = shuffle_data(c,y_datas)

In [39]:
np.isfinite(0.000000000001)

True

In [245]:
concenate_graph[np.isnan(concenate_graph)] =0.000000000001
concenate_graph[~np.isfinite(concenate_graph)] =0.000000000001

In [246]:
# c = concenate_graph[np.isnan(concenate_graph)] =0.0001
dgraph, y_data = shuffle_data(concenate_graph,y_datas)
s = StandardScaler()
s.fit(dgraph)
afters = s.transform(dgraph)
pca = PCA(n_components=62)
afterpca = pca.fit_transform(afters)
# svm = SVC(gamma = 2, C=1,random_state=0)
# score = cross_val_score(svm,afterpca,y_data,cv = 3)
# model  = make_pipeline(MinMaxScaler,PCA(n_components=62),SVC)
# print(type(model))
# model.fit(dgraph,y_data)
# with open('catlog.txt','a+') as rf:
#     rf.write(f"\n #################concatenate############ \n")
Concatenated_result =[]
for name, model in zip(names, classifiers):
#                 pca_model = make_pipeline(MinMaxScaler,PCA(n_components=100),model),
            scores = []
            try:
                score = cross_val_score(model,dgraph,y_data,cv = 3)
            except Exception:
                pass
#                 print(f'{name} ==> CV =3 ==>:',score)
            scores.append(np.mean(score))
            Concatenated_result.append(str(np.mean(score))+'+-'+str(np.std(score)))
        
                
estimators = [(name, model) for name, model in zip(names, classifiers)]
eclf1 = VotingClassifier(estimators = estimators, voting='hard')
score = cross_val_score(eclf1, concat_state_featue_sff, y_data,cv = 10)
Concatenated_result.append(str(np.mean(score))+'+-'+str(np.std(score)))

D:\Anaconda3\lib\site-packages\sklearn\svm\_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [248]:
estimators = [(name, model) for name, model in zip(names, classifiers)]
eclf1 = VotingClassifier(estimators = estimators, voting='hard')
score = cross_val_score(eclf1, dgraph, y_data,cv = 10)
Concatenated_result.append(str(np.mean(score))+'+-'+str(np.std(score)))

D:\Anaconda3\lib\site-packages\sklearn\svm\_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
D:\Anaconda3\lib\site-packages\sklearn\svm\_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
D:\Anaconda3\lib\site-packages\sklearn\svm\_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
D:\Anaconda3\lib\site-packages\sklearn\svm\_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [249]:
Concatenated_result

['0.4365079365079365+-0.04892392065848395',
 '0.5150793650793651+-0.07429012332594644',
 '0.5484126984126985+-0.019472768486325846',
 '0.5484126984126985+-0.019472768486325846',
 '0.6095238095238095+-0.15317254707066433',
 '0.42142857142857143+-0.19717391059774658',
 '0.5325396825396825+-0.012346308877860361',
 '0.5793650793650794+-0.06827242275430655',
 '0.5285714285714286+-0.09920317455237931']

# PCA each and concatenate

In [86]:
estimators = [(name, model) for name, model in zip(names, classifiers)]

In [87]:
# fdgraph =np.array(subj_graph ['Assortativity'])
# fdgraph[np.isnan(fdgraph)] =0.000000000001
# fdgraph[~np.isfinite(fdgraph)] =0.000000000001
# s = StandardScaler()
# s.fit(fdgraph)
# afters = s.transform(fdgraph)
# pca = PCA(n_components= 32)
# concenate_graph = pca.fit_transform(afters)
# with open('pca_log.txt','a+') as rf:
#     rf.write(f"\n #################scale @ pca############ \n")
#     for nmn in NMN_cat:
#         cat_dgraph = np.array(subj_graph[nmn])

#         cat_dgraph[np.isnan(cat_dgraph)] =0.000000000001
#         cat_dgraph[~np.isfinite(cat_dgraph)] =0.000000000001
#         s = StandardScaler()
#         afters = s.fit_transform(cat_dgraph)  
#         pca = PCA(n_components= 32)
#         afterpca = pca.fit_transform(afters)
#         ########  voting ########## 

#         eclf1 = VotingClassifier(estimators=estimators, voting='hard')

#         score = cross_val_score(eclf1,afterpca,y_data,cv = 10)
#         rf.write(f"==> ==> {nmn}：, Mean: {np.mean(score)}, Std：{np.std(score)}\n")
#         concenate_graph = np.concatenate([concenate_graph,afterpca],1)
# #     print('nmn',concenate_graph.shape)

D:\Anaconda3\lib\site-packages\sklearn\svm\_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
D:\Anaconda3\lib\site-packages\sklearn\svm\_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
D:\Anaconda3\lib\site-packages\sklearn\svm\_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
D:\Anaconda3\lib\site-packages\sklearn\svm\_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
D:\Anaconda3\lib\site-packages\sklearn\svm\_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
D:\Anaconda3\lib\site-packages\skle

In [79]:
with open('pca_catlog.txt','a+') as rf:
    rf.write(f"\n #################concatenate############ \n")
    for name, model in zip(names, classifiers):
#                 pca_model = make_pipeline(MinMaxScaler,PCA(n_components=100),model),
                scores = []
                try:
                    score = cross_val_score(model,concenate_graph,y_data,cv = 10)
                except Exception:
                    pass
                print(f'{name} ==> CV =3 ==>:',score)
                scores.append(np.mean(score))
                result[nmn]= scores
                rf.write(f"==> ==> {name}：, Mean: {np.mean(score)}, Std：{np.std(score)}\n")
    rf.close()

Nearest Neighbors ==> CV =3 ==>: [0.57142857 0.14285714 0.5        0.5        0.5        0.5
 0.5        0.66666667 0.33333333 0.33333333]
Linear SVM ==> CV =3 ==>: [0.57142857 0.28571429 0.5        0.5        0.33333333 0.33333333
 0.66666667 0.83333333 0.         0.16666667]
RBF SVM ==> CV =3 ==>: [0.57142857 0.57142857 0.5        0.5        0.5        0.5
 0.5        0.5        0.66666667 0.66666667]
Gaussian Process ==> CV =3 ==>: [0.57142857 0.57142857 0.5        0.5        0.5        0.5
 0.5        0.5        0.66666667 0.66666667]
Decision Tree ==> CV =3 ==>: [0.57142857 0.42857143 0.33333333 0.66666667 0.33333333 0.5
 0.83333333 0.5        0.5        0.16666667]
Random Forest ==> CV =3 ==>: [0.85714286 0.57142857 0.66666667 0.66666667 0.5        0.16666667
 0.5        0.5        0.5        0.5       ]
Neural Net ==> CV =3 ==>: [0.42857143 0.42857143 0.5        0.66666667 0.5        0.5
 0.66666667 0.83333333 0.5        0.16666667]


In [76]:
afterpca.shape
y_data.shape
score 
concenate_graph.shape

(62, 64)

In [44]:
y_train = keras.utils.to_categorical(y_data, 2)
train_ydata = y_train[:42]
test_ydata = y_train[42:]
train_xdata,test_xdata = get_train_test_data(dgraph)

In [55]:
# import keras
# from keras.layers import Input, Dense, Dropout, Activation
# model = keras.models.Sequential()
# model.add(Dense(100, use_bias=False, input_shape=(5035,), name='Dense_ly',activation='relu')) 
# model.add(Dropout(0.5))
# model.add(Dense(2, use_bias=False,activation='sigmoid')) 
# model.add(Dropout(0.4))
# model.summary()

# model.compile(loss=keras.losses.binary_crossentropy,
#               optimizer=keras.optimizers.Adam(lr=0.001),
#               metrics=['accuracy'])

# model.fit(train_xdata, train_ydata ,
#           batch_size=4,
#           epochs=500,
#           verbose=1,
#           validation_data=(test_xdata, test_ydata),
#           )

In [36]:
train_ydata 

array([0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0])